In [1]:
##################
# Libraries
##################
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from loguru import logger

In [2]:
##################
# Constants
##################
MODEL_SEED=42
ROWS_SEED=[24, 42, 206, 602, 412, 214, 754, 457, 2023, 3202]
SIZE_OF_UNLABELLED_DATA = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9]
MODELS = [
    GaussianNB(var_smoothing=0.001),
    DecisionTreeClassifier(criterion='gini', max_depth=7, min_samples_leaf=1, min_samples_split=2),
    KNeighborsClassifier(metric='euclidean', n_neighbors=3, weights='uniform'),
    RandomForestClassifier(criterion='gini', max_depth=9, min_samples_leaf=1, min_samples_split=2, n_estimators=100),
]

In [3]:
##################
# Functions
##################
def encode_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Encode dataframe using LabelEncoder"""
    labelencoder = LabelEncoder()

    for col in df.columns:
        df[col] = labelencoder.fit_transform(df[col])

    return df

def get_metrics() -> dict:
    metrics = {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'f1': f1_score,
        'roc': roc_auc_score
    }
    return metrics


def get_metrics_df(y_true, y_pred) -> pd.DataFrame:
    metrics = get_metrics()
    df = pd.DataFrame()
    for metric_name, metric in metrics.items():
        df[metric_name] = [metric(y_true, y_pred)]

    return df

def run_pipeline(X_train, X_test, y_train, y_test, size_of_unlabelled_data=0.5, model=None, rows_seed=42) -> dict:
    if model is None:
        raise ValueError("Model is None")
    
    X_train = X_train.copy()
    X_test = X_test.copy()
    y_train = y_train.copy()
    y_test = y_test.copy()

    # Unlabelled data
    rng = np.random.RandomState(rows_seed)
    random_rows_with_rng = rng.choice(X_train.index, size=int(len(X_train)*size_of_unlabelled_data), replace=False)
    y_train.loc[random_rows_with_rng] = -1

    # Scale data
    scaler = RobustScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


    self_training_model = SelfTrainingClassifier(model, verbose=False)
    self_training_model.fit(X_train, y_train)

    # Predict
    y_pred = self_training_model.predict(X_test)

    # Evaluate - Get Metrics
    metrics = get_metrics_df(y_test, y_pred)
    accuracy = metrics["accuracy"].values[0]
    precision = metrics["precision"].values[0]
    recall = metrics["recall"].values[0]
    f1 = metrics["f1"].values[0]
    roc = metrics["roc"].values[0]
    

    return {"rows_seed": rows_seed, "model": type(model).__name__, "unlabeled": size_of_unlabelled_data, "acc": accuracy, "precision": precision, "recall": recall, "f1": f1, "roc": roc}

def run_pipeline_self(X_train, X_test, y_train, y_test) -> pd.DataFrame:
    run_counter = 0
    results = []

    for model in MODELS:
        for value in SIZE_OF_UNLABELLED_DATA:
            logger.info(f'Model: {model}')
            logger.info(f'Size of unlabelled data: {value}')
            for row_seed in ROWS_SEED:
                run_counter += 1
                logger.info(f"%{run_counter} - Running pipeline for row_seed: {row_seed} and size_of_unlabelled_data: {value} and model: {model}")
                result = run_pipeline(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, size_of_unlabelled_data=value, model=model, rows_seed=row_seed)
                results.append(result)
            print('\n')

    results_rows_seed = []
    results_models = []
    results_unlabeled = []
    results_acc = []
    results_precision = []
    results_recall = []
    results_f1 = []
    results_roc = []

    for result in results:
        results_rows_seed.append(result.get('rows_seed'))
        results_models.append(result.get('model'))
        results_unlabeled.append(result.get('unlabeled'))
        results_acc.append(result.get('acc'))
        results_precision.append(result.get('precision'))
        results_recall.append(result.get('recall'))
        results_f1.append(result.get('f1'))
        results_roc.append(result.get('roc'))


    results_df = pd.DataFrame({'rows_seed': results_rows_seed, 'model': results_models, 'unlabeled': results_unlabeled, 'acc': results_acc, 'precision': results_precision, 'recall': results_recall, 'f1': results_f1, 'roc': results_roc})

    return results_df

    

In [4]:
df = pd.read_csv('../../datasets/mushrooms.csv')
df = encode_dataframe(df)


X = df.drop('class', axis=1)
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=MODEL_SEED)

In [5]:
results_df = run_pipeline_self(X_train, X_test, y_train, y_test)

2023-10-22 12:47:18.660 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:18.661 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.1
2023-10-22 12:47:18.662 | INFO     | __main__:run_pipeline_self:79 - %1 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:18.690 | INFO     | __main__:run_pipeline_self:79 - %2 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:18.716 | INFO     | __main__:run_pipeline_self:79 - %3 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:18.740 | INFO     | __main__:run_pipeline_self:79 - %4 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.1 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:18.770 | INFO     | __main__:run_pipeli

2023-10-22 12:47:19.134 | INFO     | __main__:run_pipeline_self:79 - %19 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.2 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.161 | INFO     | __main__:run_pipeline_self:79 - %20 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.2 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.190 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.190 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.3
2023-10-22 12:47:19.191 | INFO     | __main__:run_pipeline_self:79 - %21 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.215 | INFO     | __main__:run_pipeline_self:79 - %22 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.242 | INFO     | __main__:run_

2023-10-22 12:47:19.394 | INFO     | __main__:run_pipeline_self:79 - %29 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.420 | INFO     | __main__:run_pipeline_self:79 - %30 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.3 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.447 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.448 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.4
2023-10-22 12:47:19.448 | INFO     | __main__:run_pipeline_self:79 - %31 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.4 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.475 | INFO     | __main__:run_pipeline_self:79 - %32 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.4 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.499 | INFO     | __main__:run_

2023-10-22 12:47:19.654 | INFO     | __main__:run_pipeline_self:79 - %39 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.4 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.682 | INFO     | __main__:run_pipeline_self:79 - %40 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.4 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.707 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.707 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.5
2023-10-22 12:47:19.708 | INFO     | __main__:run_pipeline_self:79 - %41 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.735 | INFO     | __main__:run_pipeline_self:79 - %42 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.760 | INFO     | __main__:run_

2023-10-22 12:47:19.931 | INFO     | __main__:run_pipeline_self:79 - %49 - Running pipeline for row_seed: 2023 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.961 | INFO     | __main__:run_pipeline_self:79 - %50 - Running pipeline for row_seed: 3202 and size_of_unlabelled_data: 0.5 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.993 | INFO     | __main__:run_pipeline_self:75 - Model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:19.994 | INFO     | __main__:run_pipeline_self:76 - Size of unlabelled data: 0.6
2023-10-22 12:47:19.994 | INFO     | __main__:run_pipeline_self:79 - %51 - Running pipeline for row_seed: 24 and size_of_unlabelled_data: 0.6 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:20.022 | INFO     | __main__:run_pipeline_self:79 - %52 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.6 and model: GaussianNB(var_smoothing=0.001)
2023-10-22 12:47:20.050 | INFO     | __main__:run_

...

2023-10-22 12:48:03.759 | INFO     | __main__:run_pipeline_self:79 - %352 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:04.247 | INFO     | __main__:run_pipeline_self:79 - %353 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:04.917 | INFO     | __main__:run_pipeline_self:79 - %354 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:05.404 | INFO     | __main__:run_pipeline_self:79 - %355 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:05.896 | INFO     | __main__:run_pipeline_self:79 - %356 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.6 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:06.391 | INFO     | __main__:

2023-10-22 12:48:10.374 | INFO     | __main__:run_pipeline_self:79 - %362 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:11.015 | INFO     | __main__:run_pipeline_self:79 - %363 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:11.992 | INFO     | __main__:run_pipeline_self:79 - %364 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:13.141 | INFO     | __main__:run_pipeline_self:79 - %365 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:14.318 | INFO     | __main__:run_pipeline_self:79 - %366 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.7 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:14.969 | INFO     | __main__:

2023-10-22 12:48:18.408 | INFO     | __main__:run_pipeline_self:79 - %372 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:19.057 | INFO     | __main__:run_pipeline_self:79 - %373 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:19.874 | INFO     | __main__:run_pipeline_self:79 - %374 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:20.849 | INFO     | __main__:run_pipeline_self:79 - %375 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:21.490 | INFO     | __main__:run_pipeline_self:79 - %376 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.8 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:22.652 | INFO     | __main__:

2023-10-22 12:48:26.109 | INFO     | __main__:run_pipeline_self:79 - %382 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:26.924 | INFO     | __main__:run_pipeline_self:79 - %383 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:27.766 | INFO     | __main__:run_pipeline_self:79 - %384 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:28.595 | INFO     | __main__:run_pipeline_self:79 - %385 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:30.271 | INFO     | __main__:run_pipeline_self:79 - %386 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.85 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:31.466 | INFO     | __ma

2023-10-22 12:48:35.935 | INFO     | __main__:run_pipeline_self:79 - %392 - Running pipeline for row_seed: 42 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:36.906 | INFO     | __main__:run_pipeline_self:79 - %393 - Running pipeline for row_seed: 206 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:38.070 | INFO     | __main__:run_pipeline_self:79 - %394 - Running pipeline for row_seed: 602 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:39.245 | INFO     | __main__:run_pipeline_self:79 - %395 - Running pipeline for row_seed: 412 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:40.419 | INFO     | __main__:run_pipeline_self:79 - %396 - Running pipeline for row_seed: 214 and size_of_unlabelled_data: 0.9 and model: RandomForestClassifier(max_depth=9)
2023-10-22 12:48:41.058 | INFO     | __main__:

In [6]:
# mean and standard deviation for all metrics
results_df_with_mean_std = results_df.groupby(['model', 'unlabeled']).agg({'acc': ['mean', 'std'], 'f1': ['mean', 'std'], 'precision': ['mean', 'std'], 'recall': ['mean', 'std'], 'roc': ['mean', 'std']})
results_df_with_mean_std = results_df_with_mean_std.reset_index(level=1)
results_df_with_mean_std.columns = ["_".join(col) for col in results_df_with_mean_std.columns.values]
results_df_with_mean_std.rename(columns={'unlabeled_': 'unlabeled'}, inplace=True)
results_df_with_mean_std.reset_index(inplace=True)
results_df_with_mean_std.columns

Index(['model', 'unlabeled', 'acc_mean', 'acc_std', 'f1_mean', 'f1_std',
       'precision_mean', 'precision_std', 'recall_mean', 'recall_std',
       'roc_mean', 'roc_std'],
      dtype='object')

In [8]:
# Metrics with tabulate
from tabulate import tabulate

print(tabulate(results_df_with_mean_std, headers='keys', tablefmt='psql', showindex=False))

+------------------------+-------------+------------+-------------+-----------+-------------+------------------+-----------------+---------------+--------------+------------+-------------+
| model                  |   unlabeled |   acc_mean |     acc_std |   f1_mean |      f1_std |   precision_mean |   precision_std |   recall_mean |   recall_std |   roc_mean |     roc_std |
|------------------------+-------------+------------+-------------+-----------+-------------+------------------+-----------------+---------------+--------------+------------+-------------|
| DecisionTreeClassifier |        0.1  |   1        | 0           |  1        | 0           |         1        |     0           |      1        |  0           |   1        | 0           |
| DecisionTreeClassifier |        0.2  |   0.997785 | 0.00286005  |  0.997711 | 0.0029546   |         0.995449 |     0.00587558  |      1        |  0           |   0.997865 | 0.00275657  |
| DecisionTreeClassifier |        0.3  |   0.998338 | 0